# Utilizando somente 1 neurônio

In [34]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, classification_report, confusion_matrix

train_data = pd.read_csv('treino_sinais_vitais_com_label.csv')
test_data = pd.read_csv('treino_sinais_vitais_sem_label.csv')

X_train = train_data[["si3", "si4", "si5"]].values  # Entradas de treino
y_train_gi = train_data["gi"].values  # Gravidade (regressão) de treino

X_test = test_data[["si3", "si4", "si5"]].values  # Entradas de teste
y_test_gi = test_data["gi"].values  # Gravidade (regressão) de teste

# Normalizando os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Definindo o modelo de regressão
model_regression = Sequential(
    [Dense(1, input_dim=3, activation="linear")]  # Camada de saída com 1 neurônio
)

# Compilando o modelo
model_regression.compile(
    optimizer=Adam(learning_rate=0.03), loss="mse", metrics=["mae"]
)

model_regression.fit(X_train, y_train_gi, epochs=200, batch_size=10, verbose=1)

# Avaliando o modelo com os dados de teste
y_pred_gi = model_regression.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test_gi, y_pred_gi))
print(f"RMSE para regressão: {rmse}")

Epoch 1/200


/home/teteus/Documents/sistemas_inteligentes/trabalho02/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1942.5039 - mae: 41.1112
Epoch 2/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1512.2466 - mae: 35.8441
Epoch 3/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1233.3629 - mae: 31.6747
Epoch 4/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1027.0790 - mae: 28.7427
Epoch 5/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 803.7422 - mae: 24.7806
Epoch 6/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 693.1913 - mae: 22.7902
Epoch 7/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 549.1854 - mae: 19.9597
Epoch 8/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 428.9346 - mae: 17.1385
Epoch 9/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 380.3535 - mae: 15.9337
Epoch 10/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 323.3878 - mae: 14.6147
Epoch 11/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 264.2952 - mae: 13.1228
Epoch 12/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 241.9617 

In [47]:
results = pd.DataFrame()
results['gi'] = test_data['gi'].reset_index(drop=True)
results['gi_pred'] = y_pred_gi.flatten()

results['percentage'] = (results['gi_pred']) / results['gi']

results

,gi,gi_pred,percentage
0,40.000000,47.641941,1.191049
1,41.530426,45.720337,1.100888
2,52.730748,49.429195,0.937388
3,34.679910,41.111912,1.185468
4,69.375881,44.117523,0.635920
...,...,...,...
1495,24.494467,38.476479,1.570823
1496,65.385014,39.187809,0.599339
1497,24.530703,49.349808,2.011757
1498,61.062548,43.194950,0.707389


# Utilizando rede neural com camadas ocultas

In [45]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Carregar os dados
train_data = pd.read_csv("treino_sinais_vitais_com_label.csv")
test_data = pd.read_csv("treino_sinais_vitais_sem_label.csv")

# Separar entradas (si3, si4, si5) e saída (gi) do arquivo de treinamento
X_train = train_data[["si3", "si4", "si5"]].values
y_train_gi = train_data["gi"].values

# Entradas do arquivo de teste (sem label de gi)
X_test = test_data[["si3", "si4", "si5"]].values

# Normalização dos dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Modelo com camada oculta
model_regression = Sequential(
    [
        Dense(64, input_dim=3, activation="relu"),  # Camada oculta com 64 neurônios
        Dense(32, activation="relu"),  # Outra camada oculta com 32 neurônios
        Dense(1, activation="linear"),  # Camada de saída com 1 neurônio
    ]
)

# Compilar o modelo
model_regression.compile(
    optimizer=Adam(learning_rate=0.01), loss="mse", metrics=["mae"]
)

# Treinar o modelo
history = model_regression.fit(
    X_train, y_train_gi, epochs=100, batch_size=16, verbose=1
)

# Fazer previsões nos dados de teste
predicted_gi = model_regression.predict(X_test)

# Salvar os resultados preditos em um novo arquivo CSV
results = test_data.copy()
results["predicted_gi"] = predicted_gi
results.to_csv("resultados_preditos.csv", index=False)

print("Previsões salvas no arquivo 'resultados_preditos.csv'.")

Epoch 1/100


/home/teteus/Documents/sistemas_inteligentes/trabalho02/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 961.1302 - mae: 25.9442 
Epoch 2/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 219.2371 - mae: 11.7794
Epoch 3/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 207.4232 - mae: 11.6463
Epoch 4/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 169.6143 - mae: 10.4809
Epoch 5/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 167.0826 - mae: 10.4583
Epoch 6/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 155.4703 - mae: 9.9831
Epoch 7/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 119.0917 - mae: 8.5008
Epoch 8/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 70.9522 - mae: 6.4967
Epoch 9/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 50.7424 - mae: 5.3760
Epoch 10/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 33.7434 - mae: 4.3807
Epoch 11/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 31.9436 - mae: 4.1663
Epoch 12/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 27.1985 - mae: 3.9460
Epoch 13/100
94/94 ━━━━━

In [44]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

data = pd.read_csv("treino_sinais_vitais_com_label.csv")

X = data[["si3", "si4", "si5"]].values
y = data["yi"].values  

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.35, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model_classification = Sequential(
    [
        Dense(64, input_dim=3, activation="relu"),  # Camada oculta com 64 neurônios
        Dense(32, activation="relu"),  # Outra camada oculta com 32 neurônios
        Dense(
            4, activation="softmax"
        ),  # Camada de saída com 4 neurônios (uma para cada classe)
    ]
)

model_classification.compile(
    optimizer=Adam(learning_rate=0.01),
    loss="sparse_categorical_crossentropy", 
    metrics=["accuracy"],
)

history = model_classification.fit(
    X_train, y_train, epochs=100, batch_size=8, verbose=1
)

loss, accuracy = model_classification.evaluate(X_test, y_test, verbose=1)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

predicted_classes = model_classification.predict(X_test)
predicted_labels = label_encoder.inverse_transform(predicted_classes.argmax(axis=1))

test_results = pd.DataFrame(X_test, columns=["si3", "si4", "si5"])
test_results["true_class"] = label_encoder.inverse_transform(y_test)
test_results["predicted_class"] = predicted_labels
test_results.to_csv("classificacao_resultados.csv", index=False)

print("Resultados salvos no arquivo 'classificacao_resultados.csv'.")

Epoch 1/100


/home/teteus/Documents/sistemas_inteligentes/trabalho02/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5395 - loss: 1.0227
Epoch 2/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7136 - loss: 0.6844
Epoch 3/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7663 - loss: 0.5703
Epoch 4/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8000 - loss: 0.4915
Epoch 5/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8184 - loss: 0.4517
Epoch 6/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7956 - loss: 0.4503
Epoch 7/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8475 - loss: 0.3830
Epoch 8/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8287 - loss: 0.4268
Epoch 9/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8469 - loss: 0.3648
Epoch 10/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8352 - loss: 0.3803
Epoch 11/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8646 - loss: 0.3146
Epoch 12/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step